# 6 AutoGen - Human interaction

AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. AutoGen agents are customizable, conversable, and seamlessly allow human participation. They can operate in various modes that employ combinations of LLMs, human inputs, and tools.

17/06/2024

<img src="https://raw.githubusercontent.com/retkowsky/images/master/autogenlogo.jpg">

> Documentation: https://microsoft.github.io/autogen/

In [1]:
import autogen
import json
import openai
import os
import sys

from autogen import config_list_from_json

In [2]:
sys.version

'3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]'

In [3]:
def check_openai_version():
    """
    Check Azure Open AI version
    """
    installed_version = openai.__version__

    try:
        version_number = float(installed_version[:3])
    except ValueError:
        print("Invalid OpenAI version format")
        return

    print(f"Installed OpenAI version: {installed_version}")

    if version_number < 1.0:
        print("[Warning] You should upgrade OpenAI to have version >= 1.0.0")
        print("To upgrade, run: %pip install openai --upgrade")
    else:
        print(f"[OK] OpenAI version {installed_version} is >= 1.0.0")

In [4]:
check_openai_version()

Installed OpenAI version: 1.30.5
[OK] OpenAI version 1.30.5 is >= 1.0.0


In [5]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    #filter_dict={"tags": ["gpt-4"]},  # comment out to get all
)
# When using a single openai endpoint, you can use the following:
# config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]

In [6]:
WORK_DIR = "workdir"

os.makedirs(WORK_DIR, exist_ok=True)

## 1. Agent

In [7]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

In [8]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": None,
        "config_list": config_list,
    },
)

In [9]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith(
        "TERMINATE"),
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code.
)

In [10]:
problem_to_solve = """
I want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.
Show me the python code.
"""

In [11]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=problem_to_solve)

user_proxy (to assistant):


I want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.
Show me the python code.


--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, we can achieve this using the Pillow library in Python. Pillow provides the `Image` module for manipulating images. Here is a Python script that will resize all jpeg images in the given directory to (640,640) pixels.

In the script below, we are iterating through each file in the MYIMAGES directory, opening it, resizing it and then saving it back to the directory. 

Please ensure that Pillow library is installed in your system. If not, you can install using pip.

```shell
pip install pillow
```

Here is the complete python script you need to run.

```python
# filename: image_resizer.py

import os
from PIL import Image

IMAGE_DIRECTORY = 'MYIMAGES'
WIDTH, HEIGHT = 640, 640

def resize_image(image_path):
    with Image.open(image_path) 

ChatResult(chat_id=None, chat_history=[{'content': '\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n', 'role': 'assistant'}, {'content': "Sure, we can achieve this using the Pillow library in Python. Pillow provides the `Image` module for manipulating images. Here is a Python script that will resize all jpeg images in the given directory to (640,640) pixels.\n\nIn the script below, we are iterating through each file in the MYIMAGES directory, opening it, resizing it and then saving it back to the directory. \n\nPlease ensure that Pillow library is installed in your system. If not, you can install using pip.\n\n```shell\npip install pillow\n```\n\nHere is the complete python script you need to run.\n\n```python\n# filename: image_resizer.py\n\nimport os\nfrom PIL import Image\n\nIMAGE_DIRECTORY = 'MYIMAGES'\nWIDTH, HEIGHT = 640, 640\n\ndef resize_image(image_path):\n    with Image.open(image_path) as img:\n        img = i

> You can interact and say for example to use another library like OpenCV

In [12]:
print(user_proxy.chat_messages[assistant])

[{'content': '\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n', 'role': 'assistant'}, {'content': "Sure, we can achieve this using the Pillow library in Python. Pillow provides the `Image` module for manipulating images. Here is a Python script that will resize all jpeg images in the given directory to (640,640) pixels.\n\nIn the script below, we are iterating through each file in the MYIMAGES directory, opening it, resizing it and then saving it back to the directory. \n\nPlease ensure that Pillow library is installed in your system. If not, you can install using pip.\n\n```shell\npip install pillow\n```\n\nHere is the complete python script you need to run.\n\n```python\n# filename: image_resizer.py\n\nimport os\nfrom PIL import Image\n\nIMAGE_DIRECTORY = 'MYIMAGES'\nWIDTH, HEIGHT = 640, 640\n\ndef resize_image(image_path):\n    with Image.open(image_path) as img:\n        img = img.resize((WIDTH, HEIGHT))\n        im

In [13]:
outputfile = os.path.join(WORK_DIR, "human_interaction.json")

json.dump(user_proxy.chat_messages[assistant], open(outputfile, "w"), indent=6)

In [14]:
print("\033[1;31;34m")
with open(outputfile, "r") as f:
    print(f.read())


[
      {
            "content": "\nI want to resize all my jpeg images from the directory MYIMAGES to (640,640) pixels.\nShow me the python code.\n",
            "role": "assistant"
      },
      {
            "content": "Sure, we can achieve this using the Pillow library in Python. Pillow provides the `Image` module for manipulating images. Here is a Python script that will resize all jpeg images in the given directory to (640,640) pixels.\n\nIn the script below, we are iterating through each file in the MYIMAGES directory, opening it, resizing it and then saving it back to the directory. \n\nPlease ensure that Pillow library is installed in your system. If not, you can install using pip.\n\n```shell\npip install pillow\n```\n\nHere is the complete python script you need to run.\n\n```python\n# filename: image_resizer.py\n\nimport os\nfrom PIL import Image\n\nIMAGE_DIRECTORY = 'MYIMAGES'\nWIDTH, HEIGHT = 640, 640\n\ndef resize_image(image_path):\n    with Image.open(image_path) as 

In [15]:
!ls $WORK_DIR/*.json -lt

-rwxrwxrwx 1 root root 5251 Jun 17 09:08 workdir/human_interaction.json
